In [0]:
from keras.layers import Input, Lambda, Dense
from keras.models import Model
import keras.backend as K

# Load the TensorBoard notebook extension.
%load_ext tensorboard

# Clear any logs from previous runs
!rm -rf ./logs/ 

In [0]:
#This Code predict Assignee for Roslyn Issues with Title

import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn import preprocessing
import keras
import numpy as np

url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
embed = hub.Module(url)

data = pd.read_csv('IssueDetailsSVF_20102019_Single.csv', encoding='latin-1')

y = list(data['Assignee']) 
x = list(data['Title']) 

le = preprocessing.LabelEncoder()
le.fit(y)

categories = list(set(y))

def encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def decode(le, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)

test = encode(le,categories)

untest = decode(le, test)

x_enc = x
y_enc = encode(le, y)

#80% / 20% train / test split:

train_size = int(len(x) * .8)

x_train = np.asarray(x_enc[:train_size])
y_train = np.asarray(y_enc[:train_size])

x_test = np.asarray(x_enc[train_size:])
y_test = np.asarray(y_enc[train_size:])

In [0]:
def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)))

In [0]:
input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(UniversalEmbedding, output_shape=(512, ))(input_text)
dense = Dense(256, activation='relu')(embedding)
pred = Dense(len(categories), activation='softmax')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [0]:
# Define the Keras TensorBoard callback.
logdir= "./TestLog"
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

In [0]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    history = model.fit(x_train, y_train, epochs=3, batch_size=32,callbacks=[tensorboard_callback])
    model.save_weights('./model_SVF_Title_Assignee_Prediction.h5')

Epoch 1/3
73/73 [==============================] - 2s 33ms/step - loss: 1.8969 - acc: 0.5068



Epoch 2/3
73/73 [==============================] - 1s 12ms/step - loss: 1.6510 - acc: 0.8356
Epoch 3/3
73/73 [==============================] - 1s 11ms/step - loss: 1.4177 - acc: 0.8356


In [0]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model.load_weights('./model_SVF_Title_Assignee_Prediction.h5')  
    predicts = model.predict(x_test, batch_size=32)  

In [0]:
y_test = np.asarray(y_enc[train_size:])


y_test = decode(le, y_test)
y_preds = decode(le, predicts)

from sklearn import metrics

metrics.confusion_matrix(y_test, y_preds)

print(metrics.classification_report(y_test, y_preds))


              precision    recall  f1-score   support

   unsw-corg       0.00      0.00      0.00         2
    yuleisui       0.89      1.00      0.94        17

    accuracy                           0.89        19
   macro avg       0.45      0.50      0.47        19
weighted avg       0.80      0.89      0.85        19



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
#statistics
data['word_count'] = data['Description'].apply(lambda x: len(str(x).split()))
desc_lengths = list(data['word_count'])
print("Number of descriptions:",len(desc_lengths),
      "\nAverage word count", np.average(desc_lengths),
      "\nMinimum word count", min(desc_lengths),
      "\nMaximum word count", max(desc_lengths))

Number of descriptions: 895 
Average word count 90.37430167597765 
Minimum word count 1 
Maximum word count 1220


In [43]:
%tensorboard --logdir logs


Reusing TensorBoard on port 6006 (pid 2711), started 0:34:56 ago. (Use '!kill 2711' to kill it.)

# New Section